In [1]:
!pip install opendatasets sentence-transformers langchain-community chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.7 MB/s eta 0:00:

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/senapatirajesh/netflix-tv-shows-and-movies")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: halukcelebi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/senapatirajesh/netflix-tv-shows-and-movies


100%|██████████| 1.17M/1.17M [00:01<00:00, 1.19MB/s]

In [3]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
data_df = pd.read_csv("/content/netflix-tv-shows-and-movies/NetFlix.csv")[["title","description"]]
data_df.head()

,title,description
0,3%,In a future where the elite inhabit an island ...
1,1920,An architect and his wife move into a castle t...
2,3 Heroines,Three Indonesian women break records by becomi...
3,Blue Mountain State: The Rise of Thadland,New NFL star Thad buys his old teammates' belo...
4,Blue Planet II,This sequel to the award-winning nature series...


In [5]:
# no filtration is required

In [8]:
print(data_df.shape)
print()
print(data_df.isna().sum())

(7787, 2)

title          0
description    0
dtype: int64


In [9]:
data_df.isnull().sum()

title          0
description    0
dtype: int64

In [10]:
Embeddings = HuggingFaceEmbeddings( model_name = "sentence-transformers/all-MiniLM-L12-v2",
                                   model_kwargs ={'device': 'cuda'} )

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
loader = DataFrameLoader(data_df,page_content_column ="description")
descriptions = loader.load()

In [15]:
#descriptions

In [16]:
CHROMA_PATH = "chroma_movies_db"
chroma_db = Chroma.from_documents(
    descriptions, Embeddings, persist_directory =  CHROMA_PATH
)

In [18]:
del chroma_db

In [21]:
chroma_db = Chroma(persist_directory='/content/chroma_movies_db', embedding_function = Embeddings)
chroma_db

In [31]:
query = """
  Harry Wallcot is preparing for a bright future
"""
relevant_docs = chroma_db.similarity_search(query, k=5)

In [32]:
print( relevant_docs[0].metadata["title"])
print( relevant_docs[0].page_content )

Eddie - Strongman
British strongman Eddie Hall wants to be the strongest man in the world. This documentary reveals the grueling life choices that fuel his obsession.
